In [1]:
import pandas as pd
from bgreference import refseq
import numpy as np
import sys, os, json
from scipy import stats
from tqdm import tqdm
import scipy.stats as st
import statistics 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec
from matplotlib.lines import Line2D
import random
import matplotlib.patches as patches


Bad key "text.kerning_factor" on line 4 in
/home/migi/apps/miniconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
dfpp = pd.read_csv("data/NC_045512.2.maxBPspan250.noTetra.fold.pp.tsv", sep="\t", header=None, names=["pos", "ref", "ann"]) 
#dfpp = pd.read_csv("NC_045512.2.maxBPspan250.noTetra.fold.pp_original.tsv", sep="\t", header=None, names=["pos", "ref", "ann"]) 
print("Paired (#paired/TOTAL): "+str(len(dfpp[(dfpp["ann"]=="paired")])/len(dfpp)))
print("Unpaired (#unpaired/TOTAL): "+str(len(dfpp[(dfpp["ann"]==".")])/len(dfpp)))
print(dfpp)

spacesize=[]
for index, row in tqdm(dfpp[:-1].sort_values('pos').iterrows(), total=dfpp.shape[0]):
#for index, row in dfpp[:-1].sort_values('pos').iterrows():
    #all except last line
    incre = 0
    #ann = "."
    ann = "unpaired"
    #while ann == ".":
    while ann == "unpaired":
        incre +=1
        if not (dfpp.loc[dfpp['pos']==(row['pos']+incre), "ann"]).empty:
            ann = dfpp.loc[dfpp['pos']==(row['pos']+incre), "ann"].item()
        else:
            ann = "paired"
    spacesize.append(incre-1)

spacesize.append(0)
dfpp['spacesize']=spacesize

print(dfpp)

  0%|          | 49/29903 [00:00<01:01, 486.53it/s]

Paired (#paired/TOTAL): 0.6433802628498813
Unpaired (#unpaired/TOTAL): 0.0
         pos ref       ann
0          1   A  unpaired
1          2   U  unpaired
2          3   U  unpaired
3          4   A  unpaired
4          5   A  unpaired
...      ...  ..       ...
29898  29899   A  unpaired
29899  29900   A  unpaired
29900  29901   A  unpaired
29901  29902   A  unpaired
29902  29903   A  unpaired

[29903 rows x 3 columns]


100%|█████████▉| 29902/29903 [00:51<00:00, 580.24it/s]

         pos ref       ann  spacesize
0          1   A  unpaired          5
1          2   U  unpaired          4
2          3   U  unpaired          3
3          4   A  unpaired          2
4          5   A  unpaired          1
...      ...  ..       ...        ...
29898  29899   A  unpaired          4
29899  29900   A  unpaired          3
29900  29901   A  unpaired          2
29901  29902   A  unpaired          1
29902  29903   A  unpaired          0

[29903 rows x 4 columns]


In [5]:
afmin=0.01
afmax=0.5

def coronavirus(pos, size=1):
    return refseq("sarscov2", "0", pos, size)

def load_signature_dictionary (fjson):
    with open(fjson) as json_file: 
        return json.load(json_file) 

def get_observed_synmutation_count():
    df = pd.read_csv("data/intra/575austrian_ex.csv",sep="\t")
    #To avoid empty csqn_type (indels)
    df=df[df["csqn_type"].isin(["upstream_gene_variant","downstream_gene_variant","synonymous_variant"])]   
    df=df[(df.AF>afmin) | (df.AF<afmax)]
    list_positions = df["pos"].values
    #return np.unique(list_positions)
    return list_positions

def get_observed_mutation_count():
    df = pd.read_csv("data/intra/575austrian_ex.csv",sep="\t")
    #To avoid empty csqn_type (indels)
    df=df[df["csqn_type"].isin(["upstream_gene_variant","downstream_gene_variant","synonymous_variant","stop_gained","missense_variant","start_lost"])]   
    df=df[(df.AF>afmin) | (df.AF<afmax)]
    list_positions = df["pos"].values
    #return np.unique(list_positions)
    return list_positions

def get_observed_ntmutation_count(ref,alt):
    df = pd.read_csv("data/intra/575austrian_ex.csv",sep="\t")
    #To avoid empty csqn_type (indels)
    df=df[df["csqn_type"].isin(["upstream_gene_variant","downstream_gene_variant","synonymous_variant","stop_gained","missense_variant","start_lost"])]   
    df=df[(df["ref"]==ref) & (df["alt"]==alt)]
    df=df[(df.AF>afmin) | (df.AF<afmax)]
    list_positions = df["pos"].values
    #return np.unique(list_positions)
    return list_positions

def get_observed_ntsynmutation_count(ref,alt):
    df = pd.read_csv("data/intra/575austrian_ex.csv",sep="\t")
    #To avoid empty csqn_type (indels)
    df=df[df["csqn_type"].isin(["upstream_gene_variant","downstream_gene_variant","synonymous_variant"])]   
    df=df[(df["ref"]==ref) & (df["alt"]==alt)]
    df=df[(df.AF>afmin) | (df.AF<afmax)]
    list_positions = df["pos"].values
    #return np.unique(list_positions)
    return list_positions

def count_unpairedmutation_chunk(start, lmuta, csize):
    #Given a list of mutations, count how many unpaired there are.
    mutaunpair = dfpp[dfpp['pos'].isin(lmuta)]
    return len(mutaunpair[(mutaunpair["ann"]=="unpaired") & (mutaunpair["pos"] >= start) & (mutaunpair["pos"] <= (start+csize))])
    #return len(mutaunpair[(mutaunpair["ann"]==".") & (mutaunpair["pos"] >= start) & (mutaunpair["pos"] <= (start+csize))])

def count_unpairedSpacemutation_chunk(start, lmuta, csize):
    #Given a list of mutations, count how many unpaired there are.
    mutaunpair = dfpp[dfpp['pos'].isin(lmuta)]                  
    return len(mutaunpair[(mutaunpair["spacesize"]>3) & (mutaunpair["pos"] >= start) & (mutaunpair["pos"] <= (start+csize))])
    
    
#list of homoplasy positions
dfplas = pd.read_csv("data/GISAID_homoplasy_Mask.txt",sep="\t") 
dfplas = dfplas[['position_Start']]

N=10**4
start_position = 2
last_position = 29902
bases = set(["A","C","T","G"])
chunk_size = 29902

In [6]:
#SYN & nonSYN, ONLY C>T
jsfile="data/prob_syn_full_genome.json"
ntref="C"
ntalt="T"

simulated_probs = []
simulated_position = []
observed_mutations = get_observed_ntmutation_count(ntref,ntalt) # Get observed mutations to simulate
N_observed_mutations = len(observed_mutations)
print("Observed "+ ntref +">"+ ntalt +" mutations: "+str(N_observed_mutations))
d_signature = load_signature_dictionary(jsfile)

#list of mutations and probs.
for position in range(start_position,last_position+1,1): # Iterate over the virus genome
    ref_triplet = coronavirus(position-1,3)
    ref = ref_triplet[1]
    alts =  bases - set(ref)
    for alt in alts: # for each possible mutation of the base
        if (ref=="C" and alt == "T") and (position not in dfplas and position >55 and position < 29804):
                #syn & nonsyn C>T
                prob = d_signature[ref_triplet+"_"+alt]
                simulated_probs.append(prob)
        else:
            #homoplasy and everything else
            simulated_probs.append(0)
        simulated_position.append(position)

# Normalize simulated probabilities
simulated_probs = list(map(lambda x: x/sum(simulated_probs), simulated_probs))
            
#simulated_position #array of positions
background = np.random.choice(simulated_position, size=(N, N_observed_mutations), p=simulated_probs, replace=True) 

list_results = []
idchunk=0

for startp in tqdm(range (0,last_position,chunk_size)):
    observed_unpairedcount_chunk=count_unpairedmutation_chunk(startp, observed_mutations, chunk_size)
    # Now the simulated
    chunk_simulation = []
    for simulation in background:
        chunk_simulation.append(count_unpairedmutation_chunk(startp, simulation, chunk_size))
    mean_count_simulated = np.nanmean(chunk_simulation) # This is the mean number of simulated mutations in the chunk! let's compare it with the observed ones
    #confidence interval
    (cih,cil) = st.t.interval(0.95, len(chunk_simulation)-1, loc=np.mean(chunk_simulation), scale=st.sem(chunk_simulation))
    std=np.std(chunk_simulation)
    a = observed_unpairedcount_chunk
    b = N_observed_mutations - observed_unpairedcount_chunk
    c = mean_count_simulated
    d = N_observed_mutations - mean_count_simulated  
    if a > 0:
        odds_ratio = (a/b) / (c/d)
        u,p_value = stats.power_divergence(f_obs=[a, b], f_exp=[c, d], lambda_="log-likelihood") # perform g-test
        list_results.append([startp,startp+chunk_size,idchunk, observed_unpairedcount_chunk, mean_count_simulated, odds_ratio, u, p_value, cil, cih, std]) # save results
    idchunk+=1
    
df_results_all = pd.DataFrame(list_results,columns=["start_chunk","end_chunk","id_chunk","unpaired_observed","mean_simulated","odds_ratio","u","pvalue", "cintervL", "cintervH","stdev"]) 
df_results_all
#df_results.to_csv(outfile,sep="\t",index=False)

Observed C>T mutations: 16136


100%|██████████| 1/1 [00:27<00:00, 27.13s/it]


,start_chunk,end_chunk,id_chunk,unpaired_observed,mean_simulated,odds_ratio,u,pvalue,cintervL,cintervH,stdev
0,0,29902,0,1642,1745.0303,0.934269,6.942391,0.008418,1745.232639,1744.827961,10.321821


In [9]:
#SYN ONLY C>T
jsfile="data/prob_syn_full_genome.json"
ntref="C"
ntalt="T"

simulated_probs = []
simulated_position = []
observed_mutations = get_observed_ntsynmutation_count(ntref,ntalt) # Get observed mutations to simulate
N_observed_mutations = len(observed_mutations)
print("Observed "+ ntref +">"+ ntalt +" non-protein affecting mutations: "+str(N_observed_mutations))
d_signature = load_signature_dictionary(jsfile)

dfsyn = pd.read_csv("data/annotated_mutations_VEP.tsv",sep="\t") 
dfsyn=dfsyn[dfsyn["csqn_type"].isin(["upstream_gene_variant","downstream_gene_variant","synonymous_variant"])]
dfsyn = dfsyn[['pos', 'ref', 'alt']]

#list of mutations and probs.
for position in range(start_position,last_position+1,1): # Iterate over the virus genome
    ref_triplet = coronavirus(position-1,3)
    ref = ref_triplet[1]
    alts =  bases - set(ref)
    for alt in alts: # for each possible mutation of the base
        if not dfsyn[(dfsyn.pos==position)&(dfsyn.ref==ref)&(dfsyn.alt==alt)].empty and (ref=="C" and alt == "T") and (position not in dfplas and position >55 and position < 29804):
                #syn C>T
                prob = d_signature[ref_triplet+"_"+alt]
                simulated_probs.append(prob)
        else:
            #homoplasy and everything else
            simulated_probs.append(0)
        simulated_position.append(position)

# Normalize simulated probabilities
simulated_probs = list(map(lambda x: x/sum(simulated_probs), simulated_probs))
            
#simulated_position #array of positions
background = np.random.choice(simulated_position, size=(N, N_observed_mutations), p=simulated_probs, replace=True) 

list_results = []
idchunk=0

for startp in tqdm(range (0,last_position,chunk_size)):
    observed_unpairedcount_chunk=count_unpairedmutation_chunk(startp, observed_mutations, chunk_size)
    # Now the simulated
    chunk_simulation = []
    for simulation in background:
        chunk_simulation.append(count_unpairedmutation_chunk(startp, simulation, chunk_size))
    mean_count_simulated = np.nanmean(chunk_simulation) # This is the mean number of simulated mutations in the chunk! let's compare it with the observed ones
    #confidence interval
    (cih,cil) = st.t.interval(0.95, len(chunk_simulation)-1, loc=np.mean(chunk_simulation), scale=st.sem(chunk_simulation))
    std=np.std(chunk_simulation)
    a = observed_unpairedcount_chunk
    b = N_observed_mutations - observed_unpairedcount_chunk
    c = mean_count_simulated
    d = N_observed_mutations - mean_count_simulated  
    if a > 0:
        odds_ratio = (a/b) / (c/d)
        u,p_value = stats.power_divergence(f_obs=[a, b], f_exp=[c, d], lambda_="log-likelihood") # perform g-test
        list_results.append([startp,startp+chunk_size,idchunk, observed_unpairedcount_chunk, mean_count_simulated, odds_ratio, u, p_value, cil, cih, std]) # save results
    idchunk+=1
    
df_results_all = pd.DataFrame(list_results,columns=["start_chunk","end_chunk","id_chunk","unpaired_observed","mean_simulated","odds_ratio","u","pvalue", "cintervL", "cintervH","stdev"]) 
df_results_all
#df_results.to_csv(outfile,sep="\t",index=False)

Observed C>T non-protein affecting mutations: 5857


100%|██████████| 1/1 [00:20<00:00, 20.39s/it]


,start_chunk,end_chunk,id_chunk,unpaired_observed,mean_simulated,odds_ratio,u,pvalue,cintervL,cintervH,stdev
0,0,29902,0,479,512.672,0.928471,2.473338,0.115792,512.764972,512.579028,4.742765


In [6]:
#SYN ONLY C>T. unpair strict
jsfile="data/prob_syn_full_genome.json"
ntref="C"
ntalt="T"

simulated_probs = []
simulated_position = []
observed_mutations = get_observed_ntsynmutation_count(ntref,ntalt) # Get observed mutations to simulate
N_observed_mutations = len(observed_mutations)
print("Observed "+ ntref +">"+ ntalt +" non-protein affecting mutations: "+str(N_observed_mutations))
d_signature = load_signature_dictionary(jsfile)

dfsyn = pd.read_csv("data/annotated_mutations_VEP.tsv",sep="\t") 
dfsyn=dfsyn[dfsyn["csqn_type"].isin(["upstream_gene_variant","downstream_gene_variant","synonymous_variant"])]
dfsyn = dfsyn[['pos', 'ref', 'alt']]

#list of mutations and probs.
for position in range(start_position,last_position+1,1): # Iterate over the virus genome
    ref_triplet = coronavirus(position-1,3)
    ref = ref_triplet[1]
    alts =  bases - set(ref)
    for alt in alts: # for each possible mutation of the base
        if not dfsyn[(dfsyn.pos==position)&(dfsyn.ref==ref)&(dfsyn.alt==alt)].empty and (ref=="C" and alt == "T") and (position not in dfplas and position >55 and position < 29804):
                #syn C>T
                prob = d_signature[ref_triplet+"_"+alt]
                simulated_probs.append(prob)
        else:
            #homoplasy and everything else
            simulated_probs.append(0)
        simulated_position.append(position)

# Normalize simulated probabilities
simulated_probs = list(map(lambda x: x/sum(simulated_probs), simulated_probs))
            
#simulated_position #array of positions
background = np.random.choice(simulated_position, size=(N, N_observed_mutations), p=simulated_probs, replace=True) 

list_results = []
idchunk=0

for startp in tqdm(range (0,last_position,chunk_size)):
    observed_unpairedcount_chunk=count_unpairedmutation_chunk(startp, observed_mutations, chunk_size)
    # Now the simulated
    chunk_simulation = []
    for simulation in background:
        chunk_simulation.append(count_unpairedmutation_chunk(startp, simulation, chunk_size))
    mean_count_simulated = np.nanmean(chunk_simulation) # This is the mean number of simulated mutations in the chunk! let's compare it with the observed ones
    #confidence interval
    (cih,cil) = st.t.interval(0.95, len(chunk_simulation)-1, loc=np.mean(chunk_simulation), scale=st.sem(chunk_simulation))
    std=np.std(chunk_simulation)
    a = observed_unpairedcount_chunk
    b = N_observed_mutations - observed_unpairedcount_chunk
    c = mean_count_simulated
    d = N_observed_mutations - mean_count_simulated  
    if a > 0:
        odds_ratio = (a/b) / (c/d)
        u,p_value = stats.power_divergence(f_obs=[a, b], f_exp=[c, d], lambda_="log-likelihood") # perform g-test
        list_results.append([startp,startp+chunk_size,idchunk, observed_unpairedcount_chunk, mean_count_simulated, odds_ratio, u, p_value, cil, cih, std]) # save results
    idchunk+=1
    
df_results_all = pd.DataFrame(list_results,columns=["start_chunk","end_chunk","id_chunk","unpaired_observed","mean_simulated","odds_ratio","u","pvalue", "cintervL", "cintervH","stdev"]) 
df_results_all
#df_results.to_csv(outfile,sep="\t",index=False)

Observed C>T non-protein affecting mutations: 5857


100%|██████████| 1/1 [1:01:36<00:00, 3696.44s/it]


,start_chunk,end_chunk,id_chunk,unpaired_observed,mean_simulated,odds_ratio,u,pvalue,cintervL,cintervH,stdev
0,0,29902,0,392,415.631059,0.939066,1.472255,0.22499,415.639452,415.622666,4.282189


In [10]:
#SYN ONLY C>T flexible-spacer. simulate 10**6
jsfile="data/prob_syn_full_genome.json"
ntref="C"
ntalt="T"

simulated_probs = []
simulated_position = []
observed_mutations = get_observed_ntsynmutation_count(ntref,ntalt) # Get observed mutations to simulate
N_observed_mutations = len(observed_mutations)
print("Observed "+ ntref +">"+ ntalt +" non-protein affecting mutations: "+str(N_observed_mutations))
d_signature = load_signature_dictionary(jsfile)

dfsyn = pd.read_csv("data/annotated_mutations_VEP.tsv",sep="\t") 
dfsyn=dfsyn[dfsyn["csqn_type"].isin(["upstream_gene_variant","downstream_gene_variant","synonymous_variant"])]
dfsyn = dfsyn[['pos', 'ref', 'alt']]

#list of mutations and probs.
for position in range(start_position,last_position+1,1): # Iterate over the virus genome
    ref_triplet = coronavirus(position-1,3)
    ref = ref_triplet[1]
    alts =  bases - set(ref)
    for alt in alts: # for each possible mutation of the base
        if not dfsyn[(dfsyn.pos==position)&(dfsyn.ref==ref)&(dfsyn.alt==alt)].empty and (ref=="C" and alt == "T") and (position not in dfplas and position >55 and position < 29804):
                #syn C>T
                prob = d_signature[ref_triplet+"_"+alt]
                simulated_probs.append(prob)
        else:
            #homoplasy and everything else
            simulated_probs.append(0)
        simulated_position.append(position)

# Normalize simulated probabilities
simulated_probs = list(map(lambda x: x/sum(simulated_probs), simulated_probs))
            
#simulated_position #array of positions
background = np.random.choice(simulated_position, size=(N, N_observed_mutations), p=simulated_probs, replace=True) 

list_results = []
idchunk=0

for startp in tqdm(range (0,last_position,chunk_size)):
    observed_unpairedcount_chunk=count_unpairedSpacemutation_chunk(startp, observed_mutations, chunk_size)
    # Now the simulated
    chunk_simulation = []
    for simulation in background:
        chunk_simulation.append(count_unpairedSpacemutation_chunk(startp, simulation, chunk_size))
    mean_count_simulated = np.nanmean(chunk_simulation) # This is the mean number of simulated mutations in the chunk! let's compare it with the observed ones
    #confidence interval
    (cih,cil) = st.t.interval(0.95, len(chunk_simulation)-1, loc=np.mean(chunk_simulation), scale=st.sem(chunk_simulation))
    std=np.std(chunk_simulation)
    a = observed_unpairedcount_chunk
    b = N_observed_mutations - observed_unpairedcount_chunk
    c = mean_count_simulated
    d = N_observed_mutations - mean_count_simulated  
    if a > 0:
        odds_ratio = (a/b) / (c/d)
        u,p_value = stats.power_divergence(f_obs=[a, b], f_exp=[c, d], lambda_="log-likelihood") # perform g-test
        list_results.append([startp,startp+chunk_size,idchunk, observed_unpairedcount_chunk, mean_count_simulated, odds_ratio, u, p_value, cil, cih, std]) # save results
    idchunk+=1
    
df_results_all = pd.DataFrame(list_results,columns=["start_chunk","end_chunk","id_chunk","unpaired_observed","mean_simulated","odds_ratio","u","pvalue", "cintervL", "cintervH","stdev"]) 
df_results_all
#df_results.to_csv(outfile,sep="\t",index=False)

Observed C>T non-protein affecting mutations: 5857


100%|██████████| 1/1 [00:19<00:00, 19.18s/it]


,start_chunk,end_chunk,id_chunk,unpaired_observed,mean_simulated,odds_ratio,u,pvalue,cintervL,cintervH,stdev
0,0,29902,0,175,204.2421,0.852417,4.553183,0.032858,204.303133,204.181067,3.113469


In [7]:
#SYN ONLY C>T flexible-spacer. simulate 10**6 . unpair strict
jsfile="data/prob_syn_full_genome.json"
ntref="C"
ntalt="T"

simulated_probs = []
simulated_position = []
observed_mutations = get_observed_ntsynmutation_count(ntref,ntalt) # Get observed mutations to simulate
N_observed_mutations = len(observed_mutations)
print("Observed "+ ntref +">"+ ntalt +" non-protein affecting mutations: "+str(N_observed_mutations))
d_signature = load_signature_dictionary(jsfile)

dfsyn = pd.read_csv("data/annotated_mutations_VEP.tsv",sep="\t") 
dfsyn=dfsyn[dfsyn["csqn_type"].isin(["upstream_gene_variant","downstream_gene_variant","synonymous_variant"])]
dfsyn = dfsyn[['pos', 'ref', 'alt']]

#list of mutations and probs.
for position in range(start_position,last_position+1,1): # Iterate over the virus genome
    ref_triplet = coronavirus(position-1,3)
    ref = ref_triplet[1]
    alts =  bases - set(ref)
    for alt in alts: # for each possible mutation of the base
        if not dfsyn[(dfsyn.pos==position)&(dfsyn.ref==ref)&(dfsyn.alt==alt)].empty and (ref=="C" and alt == "T") and (position not in dfplas and position >55 and position < 29804):
                #syn C>T
                prob = d_signature[ref_triplet+"_"+alt]
                simulated_probs.append(prob)
        else:
            #homoplasy and everything else
            simulated_probs.append(0)
        simulated_position.append(position)

# Normalize simulated probabilities
simulated_probs = list(map(lambda x: x/sum(simulated_probs), simulated_probs))
            
#simulated_position #array of positions
background = np.random.choice(simulated_position, size=(N, N_observed_mutations), p=simulated_probs, replace=True) 

list_results = []
idchunk=0

for startp in tqdm(range (0,last_position,chunk_size)):
    observed_unpairedcount_chunk=count_unpairedSpacemutation_chunk(startp, observed_mutations, chunk_size)
    # Now the simulated
    chunk_simulation = []
    for simulation in background:
        chunk_simulation.append(count_unpairedSpacemutation_chunk(startp, simulation, chunk_size))
    mean_count_simulated = np.nanmean(chunk_simulation) # This is the mean number of simulated mutations in the chunk! let's compare it with the observed ones
    #confidence interval
    (cih,cil) = st.t.interval(0.95, len(chunk_simulation)-1, loc=np.mean(chunk_simulation), scale=st.sem(chunk_simulation))
    std=np.std(chunk_simulation)
    a = observed_unpairedcount_chunk
    b = N_observed_mutations - observed_unpairedcount_chunk
    c = mean_count_simulated
    d = N_observed_mutations - mean_count_simulated  
    if a > 0:
        odds_ratio = (a/b) / (c/d)
        u,p_value = stats.power_divergence(f_obs=[a, b], f_exp=[c, d], lambda_="log-likelihood") # perform g-test
        list_results.append([startp,startp+chunk_size,idchunk, observed_unpairedcount_chunk, mean_count_simulated, odds_ratio, u, p_value, cil, cih, std]) # save results
    idchunk+=1
    
df_results_all = pd.DataFrame(list_results,columns=["start_chunk","end_chunk","id_chunk","unpaired_observed","mean_simulated","odds_ratio","u","pvalue", "cintervL", "cintervH","stdev"]) 
df_results_all
#df_results.to_csv(outfile,sep="\t",index=False)

Observed C>T non-protein affecting mutations: 5857


100%|██████████| 1/1 [41:26<00:00, 2487.00s/it]


,start_chunk,end_chunk,id_chunk,unpaired_observed,mean_simulated,odds_ratio,u,pvalue,cintervL,cintervH,stdev
0,0,29902,0,111,127.465557,0.868328,2.272239,0.13171,127.470359,127.460755,2.4498
